# Elements that can be Xarrays
A few ideas on how xarray could be used to help make processing efficient.

xarray is built on Pandas, so the most of the functions that you use in Pandas are the same in xarray.  

There are two main data types in xarray: 

- `xarray.DataArray` which is similar to `pandas.Series`
- `xarray.DataSet` which is similar to `pandas.DataFrame`

Instead of `index` xarray uses `coords`

Similarly, instead of `columns` xarray uses `dims`, well sort of if no dims are given then the column names are the names of the coords.  


## Data Block
This would be a continuous time series of N points for a single channel.  The atomic element of the processing code.  The element would have methods like:
- Apply window
- Compute FFT/IFFT could use https://xrft.readthedocs.io/en/latest/index.html
- Detrend/pre-whiten
- Pick harmonic

Xarray.DataArray → 
- coords = time
- values = data
- attrs:
    - survey.id
    - station.id
    - location.latitude
    - location.longitude


In [2]:
import xarray
class Block:
    """ The atomic unit for processing time series.  Defined as a continuous time series of N points for a single channel """
    def __init__(self, time_index, data):
        self.data = xarray.DataArray(data, coords=[("time", time_index)], name="data")
        
    @property
    def window_time(self):
        """ return the time of the given window, however this is calculate, so that it can be aligned later"""
        
    def apply_window(self, window_type, window_parameters):
        """ Apply a certain window type to the data """
        window = self.get_window(window_type, window_parameters)
        return self.data * window
        
    def compute_fc(self):
        """ Compute the FFT of the given data block, could try to use <https://xrft.readthedocs.io/en/latest/index.html> """
        return FFT(self.data)
    
    

# FC Array
This would be a time series of Fourier coefficients for N channels for a single decimation level.  If the time index is the same between decimation levels, then this could be for M decimation levels.

The coordinates would be a time index that is defined by the start time of the survey and the given window length, overlap, etc. These times should correlate with the `Block.window_time`.  That way you could instantiate the FC array at the begining with zeros or NaNs, then as you calculate the FC for each window for each channel, you can then begin to fill in the FC array with values.

Probably you would have an FC array for a single channel and then collect them into a larger FC array, but they are the same object type.



In [3]:
class FCArray():
    """ An intelligent collection of FC coefficients """
    def __init__(self, time_index, channel_list):
        self.data = xarray.DataArray(data, coords=[("time", time_index), ("channels", channel_list)])